In [ ]:
import requests
import json
import pandas as pd


In [4]:
STANDINGS_URL = "https://site.api.espn.com/apis/site/v2/sports/football/nfl/standings"

def fetch_nfl_standings():
    """
    Fetches the current NFL standings from the ESPN API and returns a list of team records.
    """
    resp = requests.get(STANDINGS_URL)
    resp.raise_for_status()
    data = resp.json()

    records = []
    # The JSON structure contains 'children' for conferences/divisions
    for conference in data.get('children', []):
        for division in conference.get('groups', []):
            for team_entry in division.get('standings', []):
                team = team_entry.get('team', {})
                team_name = team.get('displayName') or team.get('shortDisplayName')
                stats = team_entry.get('stats', [])
                wins = next((int(s['value']) for s in stats if s.get('name') == 'wins'), 0)
                losses = next((int(s['value']) for s in stats if s.get('name') == 'losses'), 0)
                ties = next((int(s['value']) for s in stats if s.get('name') == 'ties'), 0)
                records.append({
                    'Team': team_name,
                    'Wins': wins,
                    'Losses': losses,
                    'Ties': ties
                })
    return records

def display_standings(records):
    """
    Pretty-prints the standings as a sorted table.
    """
    df = pd.DataFrame(records)
    # Compute win percentage
    df['Win%'] = df['Wins'] / (df['Wins'] + df['Losses'] + df['Ties'])
    # Sort by win percentage, then wins
    df = df.sort_values(by=['Win%', 'Wins'], ascending=[False, False])
    print(df.to_string(index=False))

if __name__ == '__main__':
    try:
        standings = fetch_nfl_standings()
        display_standings(standings)
    except requests.HTTPError as e:
        print(f"HTTP error occurred: {e}")
    except Exception as e:
        print(f"An error occurred: {e}")


An error occurred: 'Wins'


In [11]:
import requests

headers = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/115.0.0.0 Safari/537.36"
    )
}
params = {
    "season": 2025,
    "userRegion": "us",
    "userLanguage": "en"
}

resp = requests.get(
    "https://site.api.espn.com/apis/fitt/v1/sports/football/nfl/fpi",
    headers=headers,
    params=params
)
resp.raise_for_status()
fpi_data = resp.json()

# Print each team’s projected wins
for team in fpi_data.get("fpi", []):
    name = team["team"]["displayName"]
    wins = team["projValue"]
    print(f"{name}: {wins} projected wins")


HTTPError: 404 Client Error: Not Found for url: https://site.api.espn.com/apis/fitt/v1/sports/football/nfl/fpi?season=2025&userRegion=us&userLanguage=en

In [3]:
def update_stats(year: str ="2015"):
    """ Funcation to update the stats of the NFL Teams"""
    # 1) Define headers to mimic a browser
    headers = {
        "User-Agent": (
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
            "AppleWebKit/537.36 (KHTML, like Gecko) "
            "Chrome/113.0.0.0 Safari/537.36"
        )
    }
    url = f"https://www.espn.com/nfl/fpi/_/view/projections/season/{year}"
    print(url)

    response = requests.get(url, headers=headers)
    response.raise_for_status()

    # Print the HTML content of the page
    html_content = response.text

    # This will find *all* <table>…</table> in html_content and give you a list of DataFrames
    dfs = pd.read_html(html_content)
    df = pd.concat([dfs[0], dfs[1]], axis=1)
    return df

df = update_stats()
df


https://www.espn.com/nfl/fpi/_/view/projections/season/2015


,Team,W-L-T,PROJ W-L,PLAYOFF%,WIN DIV%,MAKE DIV%,MAKE CONF%,MAKE SB%,WIN SB%
0,Carolina Panthers,17-2-0,15.0-1.0,100.0,100.0,100.0,100.0,100.0,0.0
1,Arizona Cardinals,14-4-0,13.0-3.0,100.0,100.0,100.0,100.0,0.0,0.0
2,Cincinnati Bengals,12-5-0,12.0-4.0,100.0,100.0,0.0,0.0,0.0,0.0
3,Denver Broncos,15-4-0,12.0-4.0,100.0,100.0,100.0,100.0,100.0,100.0
4,New England Patriots,13-5-0,12.0-4.0,100.0,100.0,100.0,100.0,0.0,0.0
5,Kansas City Chiefs,12-6-0,11.0-5.0,100.0,0.0,100.0,0.0,0.0,0.0
6,Minnesota Vikings,11-6-0,11.0-5.0,100.0,100.0,0.0,0.0,0.0,0.0
7,Green Bay Packers,11-7-0,10.0-6.0,100.0,0.0,100.0,0.0,0.0,0.0
8,New York Jets,10-6-0,10.0-6.0,0.0,0.0,0.0,0.0,0.0,0.0
9,Pittsburgh Steelers,11-7-0,10.0-6.0,100.0,0.0,100.0,0.0,0.0,0.0


In [7]:
Eagles = "Philadelphia Eagles"
Cardinals = "Arizona Cardinals"
Panthers = "Carolina Panthers"
Bengals = "Cincinnati Bengals"
Colts = "Indianapolis Colts"
Jaguars = "Jacksonville Jaguars"
Dolphins = "Miami Dolphins"
Chargers = "Los Angeles Chargers"
Commanders = "Washington Commanders"
Niners = "San Francisco 49ers"
Vikings = "Minnesota Vikings"
Titans = "Tennessee Titans"
Texans = "Houston Texans"
Cowboys = "Dallas Cowboys"
Saints = "New Orleans Saints"
Chiefs = "Kansas City Chiefs"
Bears = "Chicago Bears"
Rams = "Los Angeles Rams"
Lions = "Detroit Lions"
Steelers = "Pittsburgh Steelers"
Raiders = "Las Vegas Raiders"
Jets = "New York Jets"
Falcons = "Atlanta Falcons"
Seahawks = "Seattle Seahawks"
Bills = "Buffalo Bills"
Packers = "Green Bay Packers"
Giants = "New York Giants"
Ravens = "Baltimore Ravens"
Buccaneers = "Tampa Bay Buccaneers"
Broncos = "Denver Broncos"
Patriots = "New England Patriots"
Browns = "Cleveland Browns"


player_teams = {
    "Cotter Duffy": [Eagles, Cardinals, Panthers],
    "John Taiarioli": [Bengals, Colts, Jaguars],
    "Danny Crouse": [Dolphins, Chargers, Commanders],
    "Jake Rupert": [Niners, Vikings, Titans],
    "Ben Masiowski": [Texans, Cowboys, Saints],
    "Michael Corrigan": [Chiefs, Bears, Rams],
    "Will Fredrick": [Lions, Steelers, Raiders],
    "Seth McKay": [Jets, Falcons, Seahawks],
    "Matt Wellener": [Bills, Packers, Giants],
    "Quintin Wrabley": [Ravens, Buccaneers, Browns],
}
df = pd.Series(player_teams).reset_index()
df.columns = ["player", "teams"]
df

,player,teams
0,Cotter Duffy,"[Philadelphia Eagles, Arizona Cardinals, Carol..."
1,John Taiarioli,"[Cincinnati Bengals, Indianapolis Colts, Jacks..."
2,Danny Crouse,"[Miami Dolphins, Los Angeles Chargers, Washing..."
3,Jake Rupert,"[San Francisco 49ers, Minnesota Vikings, Tenne..."
4,Ben Masiowski,"[Houston Texans, Dallas Cowboys, New Orleans S..."
5,Michael Corrigan,"[Kansas City Chiefs, Chicago Bears, Los Angele..."
6,Will Fredrick,"[Detroit Lions, Pittsburgh Steelers, Las Vegas..."
7,Seth McKay,"[New York Jets, Atlanta Falcons, Seattle Seaha..."
8,Matt Wellener,"[Buffalo Bills, Green Bay Packers, New York Gi..."
9,Quintin Wrabley,"[Baltimore Ravens, Tampa Bay Buccaneers, Cleve..."
